In [2]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from timeit import default_timer as timer
from utils import toTorchTensers, NormalizeData

base_url = r'.'

data_url = rf'{base_url}/data/Data.xlsx'
data_egypt = pd.read_excel(data_url, sheet_name='Egypt')
data_vietnam = pd.read_excel(data_url, sheet_name='Vietnam')

device = 'cpu'
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [4]:
# data
DATA = data_egypt
DATA = NormalizeData(DATA)
DATA = DATA.to_numpy()

np.random.seed(1)
np.random.shuffle(DATA)
split_point = math.floor(len(DATA) * 0.75)

train_x, train_y = DATA[:split_point, 1:], DATA[:split_point, 0:1]
test_x, test_y = DATA[split_point:, 1:], DATA[split_point:, 0:1]

[train_x, train_y, test_x, test_y] = toTorchTensers(
  train_x, train_y, test_x, test_y,
  device=device
)

In [5]:
# model
class AutoEncoder(nn.Module):
  def __init__(self, layers=[8, 6, 4]):
    super().__init__()

    in_net = 12
    out_net = 3

    hidden_net = [nn.ReLU()]
    for i in range(len(layers)-1):
      hidden_net += [
        nn.Linear(layers[i], layers[i+1]),
        nn.ReLU(),
      ]
    
    layers_reversed = layers[:]
    layers_reversed.reverse()
    hidden_net_reverse = [nn.ReLU()]
    for i in range(len(layers_reversed)-1):
      hidden_net_reverse += [
          nn.Linear(layers_reversed[i], layers_reversed[i+1]),
          nn.ReLU(),
      ]
    
    self.encoder = nn.Sequential(
      nn.Linear(in_net, layers[0]),
      *hidden_net,
      nn.Linear(layers[-1], out_net),
    )
    self.decoder = nn.Sequential(
      nn.Linear(out_net, layers[-1]),
      *hidden_net_reverse,
      nn.Linear(layers[0], in_net),
      nn.Sigmoid(),
    )

    self.current_loss = None
    self.opti = None
    self.crit = nn.MSELoss()
    self.init()
  
  def init(self):
    self.opti = optim.Adam(self.parameters(), lr=1e-3, weight_decay=1e-5)

  def forward(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded
  
  def fit(self, data_x, data_y, epoches=600, batch_size=200):
    items_len = data_x.shape[0]
    iterations = range(math.ceil(items_len/batch_size))
    start_time = timer()

    for epoche in range(epoches):
      for i in iterations:
        start = i*batch_size
        end = min((i+1)*batch_size, items_len)

        recon = self(data_x[start:end])
        loss = self.crit(recon, data_y[start:end])

        self.opti.zero_grad()
        loss.backward()
        self.opti.step()

      print(f'epoche: {epoche}, loss: {loss.item():.8f}')

    # 
    exec_time = f"{(timer() - start_time):.1f}"
    print(
      f'✅ training ended ,final loss: {loss.item():.8f}, time: {exec_time}s'
    )
    self.current_loss = loss.item()
  
  def save(self, path=None):
    if path is None:
      from datetime import datetime
      current_datetime = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
      path = rf'{base_url}/models/auto_model_{current_datetime}.pt'
    
    torch.save(
      self.state_dict(),
      path
    )
    print(f'✅ model saved as "auto_model_{current_datetime}.pt"')


In [6]:
# training
print(f"running in {train_x.device}")
model = AutoEncoder(layers=[11, 12, 5]).to(device)
model.fit(train_x, train_x, epoches=4000)


running in cpu
epoche: 0, loss: 0.15279841
epoche: 1, loss: 0.15115687
epoche: 2, loss: 0.14957385
epoche: 3, loss: 0.14802867
epoche: 4, loss: 0.14651299
epoche: 5, loss: 0.14500725
epoche: 6, loss: 0.14354928
epoche: 7, loss: 0.14210260
epoche: 8, loss: 0.14063452
epoche: 9, loss: 0.13924785
epoche: 10, loss: 0.13789809
epoche: 11, loss: 0.13647893
epoche: 12, loss: 0.13497795
epoche: 13, loss: 0.13338330
epoche: 14, loss: 0.13168178
epoche: 15, loss: 0.12985304
epoche: 16, loss: 0.12787078
epoche: 17, loss: 0.12570617
epoche: 18, loss: 0.12330294
epoche: 19, loss: 0.12059620
epoche: 20, loss: 0.11750785
epoche: 21, loss: 0.11395397
epoche: 22, loss: 0.10975537
epoche: 23, loss: 0.10455319
epoche: 24, loss: 0.09823617
epoche: 25, loss: 0.09075457
epoche: 26, loss: 0.08222470
epoche: 27, loss: 0.07310006
epoche: 28, loss: 0.06428411
epoche: 29, loss: 0.05700176
epoche: 30, loss: 0.05221588
epoche: 31, loss: 0.04981664
epoche: 32, loss: 0.04866204
epoche: 33, loss: 0.04765968
epoche: 3

In [8]:
model.save()

✅ model saved as "auto_model_07-04-2022_09-58-29.pt"


In [7]:
for input in test_x:
  recon = model(input)
  loss__ = model.crit(recon, input)

print(f'loss: {loss__:.4f}')


loss: 0.0088
